In [17]:
import os
import sys
import torch
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms, models
from PIL import Image
import os
import glob
import tqdm
from torchvision import transforms
from zmq import device
import numpy as np

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [3]:
if 0:
# This script creates symbolic links in a destination directory for images from multiple source directories.
    import os

    dest_dir = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/domain_experiment/data/continent_classification"

    src_dirs = [
        "/home/patel_zeel/kiln_compass_24/data/uttar_pradesh/images",
        "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/pakistan/pak_punjab/images",
        "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/processed_data/bangladesh/dhaka/images"
    ]

    for src in src_dirs:
        folder_name = os.path.basename(os.path.dirname(src))  # e.g., uttar_pradesh, pak_punjab, dhaka
        dest_subdir = os.path.join(dest_dir, folder_name)
        os.makedirs(dest_subdir, exist_ok=True)

        for file_name in os.listdir(src):
            src_file = os.path.join(src, file_name)
            dest_link = os.path.join(dest_subdir, file_name)

            if not os.path.exists(dest_link):
                os.symlink(src_file, dest_link)

            # Crosscheck: verify link points to correct source
            if os.path.islink(dest_link):
                target = os.readlink(dest_link)
                if os.path.abspath(target) != os.path.abspath(src_file):
                    print(f"Mismatch link: {dest_link} -> {target}")
            else:
                print(f"Missing symlink: {dest_link}")


In [4]:

if 0:
# This script splits images from multiple directories into train, val, and test sets.
    import random
    import shutil

    source_dir = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/domain_experiment/data/continent_classification"
    splits = ['train', 'val', 'test']
    split_ratio = [0.60, 0.20, 0.20]  

    for state in os.listdir(source_dir):
        state_path = os.path.join(source_dir, state)
        if not os.path.isdir(state_path):
            continue

        images = [f for f in os.listdir(state_path) if os.path.isfile(os.path.join(state_path, f))]
        random.shuffle(images)
        n = len(images)
        train_end = int(n * split_ratio[0])
        val_end = train_end + int(n * split_ratio[1])

        split_data = {
            'train': images[:train_end],
            'val': images[train_end:val_end],
            'test': images[val_end:]
        }

        for split in splits:
            split_dir = os.path.join(state_path, split)
            os.makedirs(split_dir, exist_ok=True)
            for fname in split_data[split]:
                src_file = os.path.join(state_path, fname)
                dst_file = os.path.join(split_dir, fname)
                if not os.path.exists(dst_file):
                    os.symlink(src_file, dst_file)


In [5]:
if 0:
    print("Splitting completed. Check the directories for train, val, and test splits.")

In [6]:
if 0:
# This script creates a PyTorch dataset for image rotation tasks.
    from torchvision import transforms
    from torch.utils.data import Dataset
    import random
    from PIL import Image
    import os

    class RotationDataset(Dataset):
        def __init__(self, image_paths, transform=None):
            self.image_paths = image_paths
            self.transform = transform
            self.rotations = [0, 90, 180, 270]

        def __len__(self):
            return len(self.image_paths) * 4

        def __getitem__(self, idx):
            img_idx = idx // 4
            rot_idx = idx % 4
            angle = self.rotations[rot_idx]

            image = Image.open(self.image_paths[img_idx]).convert('RGB')
            image = image.rotate(angle)

            if self.transform:
                image = self.transform(image)

            return image, rot_idx


In [20]:
if 1:
# This script defines a PyTorch model for image rotation classification.
    from torchvision import models
    import torch.nn as nn

    class RotationNet(nn.Module):
        def __init__(self, base_model):
            super().__init__()
            self.backbone = nn.Sequential(*list(base_model.children())[:-1])  # remove FC
            self.fc = nn.Linear(base_model.fc.in_features, 4)  # 4 rotation classes

        def forward(self, x):
            features = self.backbone(x)
            features = features.view(features.size(0), -1)
            out = self.fc(features)
            return out, features  # return both logits and embedding


In [8]:
if 0:
# This script prepares the dataset and dataloader for training a rotation classification model.
    images_dir = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/domain_experiment/data/continent_classification"
    import glob
    image_paths = glob.glob(os.path.join(images_dir,'uttar_pradesh','*.tif'))  # Adjust the pattern as needed
    print(f"Found {len(image_paths)} images.")

    dataset= RotationDataset(image_paths,
                            transform=transforms.Compose([
                                transforms.Resize((640, 640)),
                                transforms.ToTensor(),
                            
                            ]))





In [9]:
if 0:
# This script trains a rotation classification model using the defined dataset and model.    
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=8)

    base_model = models.resnet50(pretrained=True)
    model = RotationNet(base_model).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 10

    for epoch in tqdm.tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        for i, (images, labels) in enumerate(dataloader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs, _ = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            print(f"Epoch [{epoch+1}/{num_epochs}], Iteration [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}] Completed, Avg Loss: {epoch_loss:.4f}")
        torch.save(model.state_dict(), f"rotation_model_epoch_{epoch+1}.pth")


In [10]:
if 0:
# This script traces a PyTorch model for deployment using TorchScript.
    import timm
    import torch
    model = timm.create_model('vit_base_patch16_224', img_size=640, pretrained=True)
    dummy_input = torch.randn(1, 3, 640, 640)
    traced_model = torch.jit.trace(model, dummy_input)


In [23]:
if 1:

    model_checkpoint="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/domain_experiment/checkpoints/dhaka_resnet18_epoch_20.pth"
    base_model = models.resnet18(pretrained=True)

    model= RotationNet(base_model)
    model
    # model.load_state_dict(torch.load(model_checkpoint, map_location='cpu'))
    # model=model.to(device)
    # model.eval()
    # transform = transforms.Compose([
    #     transforms.Resize((640, 640)),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # ])
    # image_paths = glob.glob("/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/domain_experiment/data/continent_classification/dhaka/*.tif")
    # print(f"Found {len(image_paths)} images.")
    # embeddings = []
    # with torch.no_grad():
    #     for img_path in tqdm.tqdm(image_paths):
    #         img = Image.open(img_path).convert("RGB")
    #         img = transform(img).unsqueeze(0).to(device)
    #         _, feat = model(img)
    #         embeddings.append(feat.cpu().numpy())
    # embeddings = np.concatenate(embeddings, axis=0)
    # print(f"Extracted {embeddings.shape[0]} embeddings of shape {embeddings.shape[1:]}")




In [24]:
model

RotationNet(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=